<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Integrated_Gradients_and_Layer_wise_Relevance_Propagation_(LRP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

# Define Integrated Gradients function
def integrated_gradients(inputs, baseline, model, steps=50):
    # Ensure inputs and baseline require gradients
    inputs.requires_grad = True

    # Compute the scaled inputs
    grads = []
    for alpha in torch.linspace(0, 1, steps):
        scaled_input = baseline + alpha * (inputs - baseline)
        scaled_input = scaled_input.clone().detach().requires_grad_(True)

        model.zero_grad()
        output = model(scaled_input)

        # Assuming a single output; adjust if multiple outputs
        output = output[0]

        output.backward(retain_graph=True)
        grads.append(scaled_input.grad)

    # Average the gradients
    avg_grads = torch.mean(torch.stack(grads), dim=0)
    integrated_grads = (inputs - baseline) * avg_grads

    return integrated_grads

# Example model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Initialize model
model = SimpleModel()
model.eval()

# Example inputs and baseline
inputs = torch.randn(1, 10, requires_grad=True)
baseline = torch.zeros(1, 10, requires_grad=False)  # No need for gradients in baseline

# Compute integrated gradients
integrated_grads = integrated_gradients(inputs, baseline, model, steps=50)
print(integrated_grads)